# 3. Define a new score!


1 : Text query

In [ ]:
# Function to create a search query based on user input
def create_query():
    # User input for query parameters
    query_terms = input("Enter the keywords for your query: ")
    city = input("Enter the city: ")
    country = input("Enter the country: ")
    min_fees = input("Enter the minimum fees: ")
    max_fees = input("Enter the maximum fees: ")

    # Create a query dictionary with user input
    query = {
        "text": query_terms.strip(),
        "city": city.strip(),
        "country": country.strip(),
        "min_fees": float(min_fees.strip()) if min_fees.strip() else None,
        "max_fees": float(max_fees.strip()) if max_fees.strip() else None
    }

    return query

# Function to search for relevant documents based on the user query
def search_query(query, df):
    # Check if the query or query text is valid
    if not query or not query.get('text'):
        return "Please enter valid query terms."

    # Preprocess the query text similarly to the description preprocessing
    query_tokens = word_tokenize(preprocess_text(query['text']))
    query_tokens = [stemmer.stem(word) for word in query_tokens if word not in stop_words and word not in punctuation]

    # Initialize a list to store information for the relevant documents
    relevant_docs_info = []

    # Loop through each row in the DataFrame
    for index, row in df.iterrows():
        # Get the stemmed tokens from the 'descr_stem' column for the current document
        doc_tokens = set(row['descr_stem'])

        # Check if all query tokens are present in the current document
        if all(token in doc_tokens for token in query_tokens):
            # Collect relevant information for the matching document
            relevant_doc_info = row.to_dict()
            #relevant_doc_info = {
             #   'courseName': row['courseName'],
              #  'universityName': row['universityName'],
               # 'description': row['description'],
                #'url': row['url']
            #}
            relevant_docs_info.append(relevant_doc_info)

    # Create a new DataFrame from the collected information
    relevant_docs_df = pd.DataFrame(relevant_docs_info)

    # Save the results to a CSV file
    relevant_docs_df.to_csv('search_results.csv', index=False)

    return relevant_docs_df

# Execute a query using the adapted functions
user_query = create_query()
search_results = search_query(user_query, df)

# Display the search results
search_results


,url,courseName,universityName,facultyName,fullTime,description,startDate,fees,modality,duration,city,country,administator,preprocessed_desc,descr_stem,numeric_fee,currency_code,fees_usd,text,new_score
0,https://www.findamasters.com/masters-degrees/c...,Applied Computer Science and Artificial Intell...,University of Bradford,Faculty of Engineering & Digital Technologies,True,Computer science is the foundation of many exc...,"September, January",FeesPlease see the university website for furt...,MSc,1 Year Full Time / 2 Years Part Time,Bradford,United Kingdom,On Campus,computer science foundation many exciting topi...,"[comput, scienc, foundat, mani, excit, topic, ...",NaN,USD,NaN,comput scienc foundat mani excit topic technol...,0.0
1,https://www.findamasters.com/masters-degrees/c...,Astronomy MSc,University of Groningen,Science and Engineering,True,A Master's degree in Astronomy is a gateway in...,September,FeesPlease see the university website for furt...,MSc,2 years full time,Groningen,Netherlands,On Campus,master degree astronomy gateway wide world sci...,"[master, degre, astronomi, gateway, wide, worl...",NaN,USD,NaN,master degre astronomi gateway wide world scie...,0.0
2,https://www.findamasters.com/masters-degrees/c...,Computational Geoscience - MSc,University of Glasgow,College of Science and Engineering,True,The Masters in Computational Geoscience explor...,September,FeesPlease see the university website for furt...,MSc,"1 year full-time, 2 years part-time",Glasgow,United Kingdom,On Campus,masters computational geoscience explores mode...,"[master, comput, geoscienc, explor, model, nat...",NaN,USD,NaN,master comput geoscienc explor model natur wor...,0.0
3,https://www.findamasters.com/masters-degrees/c...,Computer Science,"St Mary’s University, Twickenham","Faculty of Sport, Health, and Applied Science",True,Join the ever-expanding community of computer ...,September,NaN,MSc,1 year,London,United Kingdom,NaN,join community computer scientists tackle issu...,"[join, commun, comput, scientist, tackl, issu,...",NaN,USD,NaN,join commun comput scientist tackl issu comput...,0.0
4,https://www.findamasters.com/masters-degrees/c...,Data Analytics - MSc,University of Glasgow,College of Science and Engineering,True,We also offer a part-timeonline MSc in Data An...,September,FeesPlease see the university website for furt...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,also offer msc data analyticsdata becoming eve...,"[also, offer, msc, data, analyticsdata, becom,...",NaN,USD,NaN,also offer msc data analyticsdata becom ever i...,0.0
5,https://www.findamasters.com/masters-degrees/c...,Data Analytics - MSc/PgDip/PgCert (Online),University of Glasgow,College of Science and Engineering,False,While the amount of data being produced is pro...,September,FeesPlease see the university website for furt...,"PGCert,MSc,PGDip",See Programme Description,Glasgow,United Kingdom,NaN,amount data produced proliferating staggering ...,"[amount, data, produc, prolifer, stagger, rate...",NaN,USD,NaN,amount data produc prolifer stagger rate skill...,0.0
6,https://www.findamasters.com/masters-degrees/c...,Data Analytics and Human Resource Management -...,University of Leeds,Leeds University Business School,True,Developed with world-leading experts in data a...,September,"FeesUK: £16,750 (Total)International: £30,250 ...",MSc,1 year full time,Leeds,United Kingdom,On Campus,developed experts data analytics hr profession...,"[develop, expert, data, analyt, hr, profession...",750.0,USD,750.0,develop expert data analyt hr profession innov...,0.0
7,https://www.findamasters.com/masters-degrees/c...,Data Science - MSc,University of Sussex,School of Mathematical and Physical Sciences,True,Do you wish to work in industry or plan to pur...,September,FeesFind the specific fees for your chosen pro...,MSc,"1 year full-time, 2 years part-time",Brighton,United Kingdom,On Campus,wish work industry plan pursue study intereste...,"[wish, work, industri, plan, purs

Here our keyword query was "Data science" so as a result we have all the data science degree available. It is also possible to filter the results based on a city a country and the fees.

2 : New score

In [ ]:
# Function to calculate a new scoring mechanism for documents based on user query
def new_scoring_function(documents, user_query):
    # Create a new column for the new score and initialize it to 0.0
    documents["new_score"] = 0.0

    # Iterate through each document in the DataFrame
    for i in range(len(documents)):
        # Extract relevant information from the user query and document
        query_match_score = documents.loc[i, "Similarity"]  # Replace with the correct column or calculation
        city_query = user_query.get('city', '')
        country_query = user_query.get('country', '')
        fee_range = (user_query.get('min_fees', 0), user_query.get('max_fees', float('inf')))

        # Calculate scores for city, country, and fees
        city_score = int(city_query.lower() in documents.loc[i, "city"].lower())
        country_score = int(country_query.lower() in documents.loc[i, "country"].lower())
        
        # Handle potential errors when converting fees to float
        try:
            fee_score = int(fee_range[0] <= float(documents.loc[i, "fees_usd"]) <= fee_range[1])
        except (ValueError, TypeError):
            fee_score = 0

        # Define weights for different components of the new score
        weights = {
            'query_match': 0.6,
            'city': 0.1,
            'country': 0.2,
            'fees': 0.1
        }

        # Calculate the new score for the document
        new_score = (query_match_score * weights['query_match'] +
                     city_score * weights['city'] +
                     country_score * weights['country'] +
                     fee_score * weights['fees'])

        # Assign the new score to the corresponding row in the DataFrame
        documents.at[i, "new_score"] = new_score

    return documents


The newly introduced scoring function does not outperform the similarity score. The key distinction lies in the fact that the similarity score relies solely on the terms within the description, whereas the new scoring function assigns distinct weights to each criterion. This implies that the importance of specific criteria may fluctuate depending on the context or domain.

# 4. Visualising the most relevant MSc degrees
1. Merging query result and coordinates
2. Visualising the query result on a map

In [1]:
import general_functions
import collecting_coords

## 1. Merging query result and coordinates
We have collected the coordinates of the universities by geo-coding (using nominatim) where possible. Where it wasn't possible we either collected the coordinates of the cities. In one case we had to hardcode the coordinates as ultima ratio.
You can look up the explicit code in collecting_coords.py.
Next we just get the search_results from abve, add the coordinates and show the first three examples.

In [6]:
query_result = search_results.copy()
query_result = collecting_coords.add_coords(query_result)
query_result[['courseName','universityName','lat','lng']].head(3)

,courseName,universityName,lat,lng
0,Applied Computer Science and Artificial Intell...,University of Bradford,53.791277,-1.766611
1,Astronomy MSc,University of Groningen,53.219442,6.562986
2,Computational Geoscience - MSc,University of Glasgow,55.872315,-4.289219


## 2. Visualising the query result on a map
You will see a map created by folium with adjusted zoom-level. The locations indicated are the coordinates of the universities of the courses listed in the query_result. If the query_result contains multiple courses of the same university, the courses are enumerated. You can see the courses together with possible fees by clicking on the marked locations.
Further each location is linked to the other locations of the query result. Moving the cursor over this line yields information about the universities connected by this line and the distance between them.

In [7]:
import map_visualisation
map_visualisation.show_map(map_visualisation.beautify(query_result))